# Evaluation for Supervised Learning Cache Replacement Policy

## Install Dependency

In [1]:
import sys
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm 
from collections import Counter, deque, defaultdict
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
import pickle


## Block Cache Model

### Global Variable

In [2]:
# Maximum block number
maxpos = 1000000000000

# Number of features (Recency, Frequency, Block No.)
num_params = 3

# Cache Size
cache_size = 100

# Sequence Length
sequence_length = 5

# Storage
sequence_data = np.zeros((sequence_length, cache_size, num_params))

# How often to store the data (Removed currently)
sampling_freq = cache_size

# How many % of cache to use
eviction = int(0.7 * cache_size)  

# Results
lruCorrect = 0
lruIncorrect = 0

lfuCorrect = 0
lfuIncorrect = 0

# Variables
X = np.array([], dtype=np.int64).reshape(0,num_params)
Y = np.array([], dtype=np.int64).reshape(0,1)

### Data Preprocessing and Data Construction

In [3]:
# Taken from Shehbaz
def get_recency(lru, cache):
    recency = []
    recency_dict = defaultdict(int)
    
    # Compute the recency order of each page in cache
    for time in range(len(lru)):
        recency_dict[lru[time]] = time + 1
        
    for block in cache:
        recency.append(recency_dict[block])

    return recency

def get_frequency(lfu, cache):
    frequency = []
    
    for block in cache:
        frequency.append(lfu[block])
    return frequency

def normalize_columns(input):
    return normalize(input, axis=0)

In [4]:
def get_single_length_input(lfu, lru, cache, preprocess_func, cache_size):
    input_recency = get_recency(lru, cache)
    input_frequency = get_frequency(lfu, cache)
    input_block_num = cache[:]
    
    if len(input_recency) < cache_size:
        for i in range(0, cache_size - len(input_recency)):
            input_recency.append(0)
            input_frequency.append(0)
            input_block_num.append(0)
    
    # Columns: recency, frequency, block number
    # Row: cache location
    raw_input = np.column_stack((input_recency, input_frequency, input_block_num))
    
    return preprocess_func(raw_input)


SEQ_DIM = 0
def get_multiple_length_input(sequence_length, prev_inputs, lfu, lru, cache, preprocess_func, cache_size):
    assert prev_inputs.shape[SEQ_DIM] == sequence_length
    current_input = get_single_length_input(lfu, lru, cache, preprocess_func, cache_size)
    return np.vstack((prev_inputs[1:], current_input[None]))
    

def get_outputs(cache, delete):
    assert(len(cache) == len(delete))
    Y_current = []
    KV_sorted = Counter(delete)
    evict_dict = dict(KV_sorted.most_common(eviction))
    assert(len(evict_dict) == eviction)
    all_vals = evict_dict.values()
    for e in cache:
        if e in evict_dict.values():
            Y_current.append(1)
        else:
            Y_current.append(0)

    assert(Y_current.count(1) == eviction)
    assert((set(all_vals)).issubset(set(cache)))
    return Y_current


def store_pair(request_time, seq_len, seq_data, lfu, lru, cache, deleted, cache_size, preprocess_func):
    global X,Y,Z
    cache = list(cache)
    Y_current = get_outputs(cache, deleted)
    
    Y = np.vstack((Y, np.array(Y_current)[None]))
    X = np.vstack((X, seq_data[None]))
    Z = np.vstack((Z, request_time))

    assert(Y_current.count(1) == eviction)
    return Y_current

### Some helper functions

In [5]:
def lruPredict(C,LRUQ,Y_OPT):
    global lruCorrect, lruIncorrect
    Y_current = []
    KV = defaultdict(int)
    for i in range(len(LRUQ)):
        KV[LRUQ[i]] = len(LRUQ) - i
    KV_sorted = Counter(KV)
    evict_dict = dict(KV_sorted.most_common(eviction))
    for e in C:
        if e in evict_dict:
            Y_current.append(1)
        else:
            Y_current.append(0)
    for i in range(len(Y_current)):
        if Y_current[i] is Y_OPT[i]:
            lruCorrect+=1
        else:
            lruIncorrect+=1
    return Y_current

# returns sequence of blocks in prioirty order

def Y_getBlockSeq(Y_pred_prob):
    x = []
    for i in range(len(Y_pred_prob)):
        x.append(Y_pred_prob[i][0])
    x = np.array(x)
    idx = np.argsort(x)
    idx = idx[:eviction]
    return idx


def Y_getMinPredict(Y_pred_prob):
    x = []
    for i in range(len(Y_pred_prob)):
        x.append(Y_pred_prob[i][0])
    x = np.array(x)
    idx = np.argpartition(x, eviction)
    
    Y_pred = np.zeros(len(Y_pred_prob), dtype=int)
    for i in range(eviction):
        Y_pred[idx[i]] = 1
    assert(Counter(Y_pred)[1] == eviction)
    return Y_pred


def lfuPredict(C,LFUDict,Y_OPT):
    global lfuCorrect, lfuIncorrect
    Y_current = []
    KV = defaultdict()
    for e in C:
        KV[e] = LFUDict[e]
    KV_sorted = Counter(KV)
    evict_dict = dict(KV_sorted.most_common(eviction))
    for e in C:
        if e in evict_dict:
            Y_current.append(1)
        else:
            Y_current.append(0)
    for i in range(len(Y_current)):
        if Y_current[i] is Y_OPT[i]:
            lfuCorrect+=1
        else:
            lfuIncorrect+=1
    return Y_current

# return "eviction" blocks that are being accessed furthest
# from the cache that was sent to us.

def getY(C,D):
    assert(len(C) == len(D))
    Y_current = []
    KV_sorted = Counter(D)
    evict_dict = dict(KV_sorted.most_common(eviction))
    assert(len(evict_dict) == eviction)
    all_vals = evict_dict.values()
    for e in C:
        if e in evict_dict.values():
            Y_current.append(1)
        else:
            Y_current.append(0)
    #print (Y_current.count(1))
    assert(Y_current.count(1) == eviction)
    assert((set(all_vals)).issubset(set(C)))
    return Y_current

def getLFURow(LFUDict, C):
    x_lfurow = []
    for e in C:
        x_lfurow.append(LFUDict[e])
    norm = x_lfurow / np.linalg.norm(x_lfurow)
    return norm
    
def getLRURow(LRUQ, C):
    x_lrurow = []
    KV = defaultdict(int)
    for i in range(len(LRUQ)):
        KV[LRUQ[i]] = i
    for e in C:
        x_lrurow.append(KV[e])
    norm = x_lrurow / np.linalg.norm(x_lrurow)
    return norm

def normalize(feature, blocks):
    x_feature = []
    for i in range(len(blocks)):
        x_feature.append(feature[blocks[i]])
    return x_feature / np.linalg.norm(x_feature)

def getX(LRUQ, LFUDict, C):
#def getX(LRUQ, LFUDict, C, CacheTS, CachePID):   
    X_lfurow = getLFURow(LFUDict, C)
    X_lrurow = getLRURow(LRUQ, C)
    X_bno    = C / np.linalg.norm(C)
#     X_ts     = normalize(CacheTS, C)
#     X_pid    = normalize(CachePID, C)
    return (np.column_stack((X_lfurow, X_lrurow, X_bno)))
    
    
def populateData(LFUDict, LRUQ, C, D):
#def populateData(LFUDict, LRUQ, C, D, CacheTS, CachePID):
    global X,Y
    C = list(C)
    Y_current = getY(C, D)
    #X_current = getX(LRUQ, LFUDict, C, CacheTS, CachePID)
    X_current = getX(LRUQ, LFUDict, C)

    Y = np.append(Y, Y_current)
    X = np.concatenate((X,X_current))
    assert(Y_current.count(1) == eviction)
    return Y_current

### Belady Optimal Algorithm (From Shehbaz)

In [12]:
def belady_opt(blocktrace, frame):
    global maxpos, sequence_data, sequence_length
    
    optimal = defaultdict(deque)
    deleted = defaultdict(int)
    lfu = defaultdict(int)
    lru = []

    # Build the whole index for finding optimal eviction ordering
    for request_time, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        optimal[block].append(request_time)

    hit, miss = 0, 0

    cache = []
    
    for request_time, block in enumerate(tqdm(blocktrace, desc="OPT")):
        old_seq = sequence_data[:]
        # increase frequency count
        lfu[block] +=1

        # Remove the block i at time step j from the index
        if len(optimal[block]) is not 0 and optimal[block][0] == request_time:
            optimal[block].popleft()

        
        if block in cache:
            # Cache Hit
            # Update block to MRU position
            hit += 1
            lru.remove(block)
            lru.append(block)
            
            assert request_time in deleted
            
            del deleted[request_time]
            if len(optimal[block]) is not 0:
                deleted[optimal[block][0]] = block
                optimal[block].popleft()
            else:
                deleted[maxpos] = block
                maxpos -= 1
            sequence_data = get_multiple_length_input(sequence_length, sequence_data, lfu, lru, cache, lambda x: x, cache_size)
        else:
            # Cache Miss
            miss += 1
            if len(cache) == frame:
                # Cache is full
                assert(len(deleted) == frame)
                evictpos = max(deleted)
                
                #if (seq_number % sampling_freq +1 == sampling_freq):
                y_opt = store_pair(request_time, sequence_length, sequence_data, lfu, lru, cache, deleted, cache_size, lambda x: x)
    
                cache[cache.index(deleted[evictpos])] = block
                lru.remove(deleted[evictpos])
                del deleted[evictpos]
            else:
                # Cache isn't full
                cache.append(block)                
            
            # Add the candidate victim page to the j'th time step
            if len(optimal[block]) is not 0:
                deleted[optimal[block][0]] = block
                optimal[block].popleft()
            else:
                deleted[maxpos] = block
                maxpos -= 1
            lru.append(block)
            sequence_data = get_multiple_length_input(sequence_length, sequence_data, lfu, lru, cache, lambda x: x, cache_size)


    hitrate = hit / (hit + miss)

    return hitrate

## Generate training or testing (evaluating) data

### online.cs.fiu.edu-09 (casa-09)
percentage:0.1

In [9]:
# train = "FIU_trace/cheetah.cs.fiu.edu-110108-113008.2.blkparse"
train = "online.cs.fiu.edu-110108-113008.9.blkparse"

df = pd.read_csv(train, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']

trainBlockTrace = df['blockNo'].tolist()
trainBlockTrace = trainBlockTrace[:int(len(trainBlockTrace)*0.1)]
# trainBlockTrace = trainBlockTrace[:300]
# df.head()
len(trainBlockTrace)

47836

In [10]:
test_casa09 = "online.cs.fiu.edu-110108-113008.9.blkparse"

df = pd.read_csv(test_casa09, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']

testBlockTrace_casa09 = df['blockNo'].tolist()
testBlockTrace_casa09 = testBlockTrace_casa09[:int(len(testBlockTrace_casa09)*0.1)]
# trainBlockTrace = trainBlockTrace[:300]
len(testBlockTrace)

47836

In [30]:
X = np.array([], dtype=np.int64).reshape(0, sequence_length, cache_size, num_params)
Y = np.array([], dtype=np.int64).reshape(0, cache_size)
Z = np.array([], dtype=np.int32).reshape(0, 1)
trainHitrate = belady_opt(trainBlockTrace, cache_size)
X_train = X
Y_train = Y

# training_data_file = 'fiu_01.pkl'
training_data_file = 'online_09.pkl'
with open(training_data_file, 'wb+') as f:
    pickle.dump([X, Y, Z], f)

### online.cs.fiu.edu-12 (casa-12)
percentage:0.05

In [17]:
train = "online.cs.fiu.edu-110108-113008.12.blkparse"

df = pd.read_csv(train, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']

trainBlockTrace = df['blockNo'].tolist()
trainBlockTrace = trainBlockTrace[:int(len(trainBlockTrace)*0.05)]
# trainBlockTrace = trainBlockTrace[:300]
len(trainBlockTrace)

52358

In [19]:
test_casa12 = "online.cs.fiu.edu-110108-113008.12.blkparse"

df = pd.read_csv(test_casa12, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']

testBlockTrace_casa12 = df['blockNo'].tolist()
testBlockTrace_casa12 = testBlockTrace_casa12[:int(len(testBlockTrace_casa12)*0.05)]
# trainBlockTrace = trainBlockTrace[:300]
len(testBlockTrace)

104716

In [32]:
X = np.array([], dtype=np.int64).reshape(0, sequence_length, cache_size, num_params)
Y = np.array([], dtype=np.int64).reshape(0, cache_size)
Z = np.array([], dtype=np.int32).reshape(0, 1)
trainHitrate = belady_opt(trainBlockTrace, cache_size)

### webmail+online.cs.fiu.edu-12 (cheetah-12)
percentage:0.025

In [13]:
train = "webmail+online.cs.fiu.edu-110108-113008.12.blkparse"

df = pd.read_csv(train, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']

trainBlockTrace = df['blockNo'].tolist()
trainBlockTrace = trainBlockTrace[:int(len(trainBlockTrace)*0.025)]
# trainBlockTrace = trainBlockTrace[:300]
len(trainBlockTrace)

52370

In [34]:
X = np.array([], dtype=np.int64).reshape(0, sequence_length, cache_size, num_params)
Y = np.array([], dtype=np.int64).reshape(0, cache_size)
Z = np.array([], dtype=np.int32).reshape(0, 1)
trainHitrate = belady_opt(trainBlockTrace, cache_size)
X_train = X
Y_train = Y

# training_data_file = 'fiu_01.pkl'
training_data_file = 'webmail_12.pkl'
with open(training_data_file, 'wb+') as f:
    pickle.dump([X, Y, Z], f)

### webmail+online.cs.fiu.edu-16 (cheetah-16)
percentage:0.05

In [35]:
train = "webmail+online.cs.fiu.edu-110108-113008.16.blkparse"

df = pd.read_csv(train, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']

trainBlockTrace = df['blockNo'].tolist()
trainBlockTrace = trainBlockTrace[:int(len(trainBlockTrace)*0.05)]
# trainBlockTrace = trainBlockTrace[:300]
len(trainBlockTrace)

47275

In [36]:
X = np.array([], dtype=np.int64).reshape(0, sequence_length, cache_size, num_params)
Y = np.array([], dtype=np.int64).reshape(0, cache_size)
Z = np.array([], dtype=np.int32).reshape(0, 1)
trainHitrate = belady_opt(trainBlockTrace, cache_size)
X_train = X
Y_train = Y

# training_data_file = 'fiu_01.pkl'
training_data_file = 'webmail_16.pkl'
with open(training_data_file, 'wb+') as f:
    pickle.dump([X, Y, Z], f)

## LRU, LFU and Belady methods

### Belady (OPT)

In [ ]:
def belady_opt(blocktrace, frame):
    global maxpos
    
    OPT = defaultdict(deque)
    D = defaultdict(int)
    LFUDict = defaultdict(int)
    LRUQ = []
    #CacheTS = defaultdict(int)
    #CachePID = defaultdict(int)
    for i, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        OPT[block].append(i)

    hit, miss = 0, 0

    C = []
    #count=0
    #seq_number = 0
#     f = open("Belady_online12_evaluation.txt","w+")
    f = open("Belady_online09_evaluation.txt","w+")
    for seq_number, block in enumerate(tqdm(blocktrace, desc="OPT")):
        LFUDict[block] +=1

        if len(OPT[block]) is not 0 and OPT[block][0] == seq_number:
            OPT[block].popleft()
        #CacheTS [blocktrace[seq_number]] = timestamp[seq_number]
        #CachePID [blocktrace[seq_number]] = pid[seq_number]
        if block in C:
            hit+=1
            LRUQ.remove(block)
            LRUQ.append(block)
            assert( seq_number in D)
            del D[seq_number]
            if len(OPT[block]) is not 0:
                D[OPT[block][0]] = block
                OPT[block].popleft()
            else:
                D[maxpos] = block
                maxpos -= 1
        else:
            miss+=1
            if len(C) == frame:
                assert(len(D) == frame)
                evictpos = max(D)
                
                if (seq_number % sampling_freq +1 == sampling_freq):
                    #Y_OPT = populateData(LFUDict, LRUQ, C, D, CacheTS, CachePID)
                    Y_OPT = populateData(LFUDict, LRUQ, C, D)
                    lruPredict(C,LRUQ,Y_OPT)
                    lfuPredict(C,LFUDict,Y_OPT)
                
                C[C.index(D[evictpos])] = block
                LRUQ.remove(D[evictpos])
                #del CacheTS [D[evictpos]]
                #del CachePID [D[evictpos]]
                del D[evictpos]
            else:
                C.append(block)
                
            if len(OPT[block]) is not 0:
                D[OPT[block][0]] = block
                OPT[block].popleft()
            else:
                D[maxpos] = block
                maxpos -= 1
            LRUQ.append(block)
        if(seq_number%2000==0):
#             f.write("%d\n" % seq_number)
            f.write("%f\n" % (hit/(hit + miss)))
    f.close()
    hitrate = hit / (hit + miss)
#     f = open("Belady_online12_evaluation.txt","a+")
    f = open("Belady_online09_evaluation.txt","a+")
#     f.write("%d\n" % seq_number)
    f.write("%f\n" % hitrate)
    f.close()
    return hitrate

### LFU

In [ ]:
def LFU(blocktrace, frame):
    
    cache = set()
    cache_frequency = defaultdict(int)
    frequency = defaultdict(int)
    
    hit, miss = 0, 0
    
#     f_file = open("LFU_online12_evaluation.txt","w+")
    f_file = open("LFU_online09_evaluation.txt","w+")
    for block in tqdm(blocktrace, leave=False):
        frequency[block] += 1
        
        if block in cache:
            hit += 1
            cache_frequency[block] += 1
        
        elif len(cache) < frame:
            cache.add(block)
            cache_frequency[block] += 1
            miss += 1

        else:
            e, f = min(cache_frequency.items(), key=lambda a: a[1])
            cache_frequency.pop(e)
            cache.remove(e)
            cache.add(block)
            cache_frequency[block] = frequency[block]
            miss += 1
            
        if((hit+miss)%2000==0):
#             f_file.write("%d\n" % (hit+miss))
            f_file.write("%f\n" % (hit/(hit + miss)))
    
    f_file.close()
    hitrate = hit / ( hit + miss )
#     f_file = open("LFU_online12_evaluation.txt","a+")
    f_file = open("LFU_online09_evaluation.txt","a+")
#     f_file.write("%d\n" % (hit+miss))
    f_file.write("%f\n" % hitrate)
    f_file.close()
    return hitrate

### LRU

In [ ]:
def LRU(blocktrace, frame):
    
    cache = set()
    recency = deque()
    hit, miss = 0, 0
    
#     f = open("LRU_online12_evaluation.txt","w+")
    f = open("LRU_online09_evaluation.txt","w+")
    for block in tqdm(blocktrace, leave=False):
        
        if block in cache:
            recency.remove(block)
            recency.append(block)
            hit += 1
            
        elif len(cache) < frame:
            cache.add(block)
            recency.append(block)
            miss += 1
            
        else:
            cache.remove(recency[0])
            recency.popleft()
            cache.add(block)
            recency.append(block)
            miss += 1
        
        if((hit+miss)%2000==0):
#             f.write("%d\n" % (hit+miss))
            f.write("%f\n" % (hit/(hit + miss)))
    f.close()
    hitrate = hit / (hit + miss)
#     f = open("LRU_online12_evaluation.txt","a+")
    f = open("LRU_online09_evaluation.txt","a+")
#     f.write("%d\n" % (hit+miss))
    f.write("%f\n" % hitrate)
    f.close()
    return hitrate

### Evaluation with Belady, LFU and LRU methods

#### Casa-09

In [ ]:
belady_opt(testBlockTrace_casa09, cache_size)

In [ ]:
LFU(testBlockTrace_casa09, cache_size)

In [ ]:
LRU(testBlockTrace_casa09, cache_size)

#### Casa-12

In [ ]:
belady_opt(testBlockTrace_casa12, cache_size)

In [ ]:
LFU(testBlockTrace_casa12, cache_size)

In [ ]:
LRU(testBlockTrace_casa12, cache_size)